## Headers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from string import punctuation
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=nltk.stem.WordNetLemmatizer()
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import time
import datetime

## Loading Dataset

In [3]:
Dp=pd.read_csv("I:/FLAIRMINDS/Product_matching/product-data-category.csv")
Dp

,id,product_id,website_id,normalized_leaf_category_id,department_name,iksula_category_identifier,product_url,product_title,model_number,manufacturing_brand
0,1311275,48171894,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09KQBQLZQ,Multifunctional Air Compressor,NaN,Generic
1,1310580,43184194,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B06Y3W2676,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",312-071B,Beverage Air
2,1310903,46036109,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08NK54DG2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,NaN,Vbestlife
3,1311137,47687124,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08FXSQTCB,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,NaN,RYOBI
4,1311979,52530864,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08P5KFNRD,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,NaN,Fafeicy
...,...,...,...,...,...,...,...,...,...,...
4022,1311446,48873158,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B01L8Q30SI,Professional Portable Airbrush Black Mini Air ...,‎31CPR010,Generic
4023,1310830,45490321,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B077XB9745,Update -3 Set Pressure Electric High Pressure ...,NaN,TECHTONGDA
4024,1311723,49827612,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09D7HK835,20V Tire Inflator Air Compressor Portable Bund...,NaN,WORKSITE
4025,1311219,47891808,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09DCK5H1M,HPDAVV High Pressure Air Compressor Oil-Free 1...,NaN,HPDAVV


## product_title dataframe

In [4]:
title=pd.DataFrame(Dp["product_title"])

In [5]:
title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027 entries, 0 to 4026
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_title  4026 non-null   object
dtypes: object(1)
memory usage: 31.6+ KB


In [6]:
title.isnull().values.any()

True

In [7]:
print(title.isnull().sum().sum())
title=title.dropna()

1


In [8]:
## checking if stopwords are available or not

stopWords = set(stopwords.words('english'))
def remove_stopwords(text):
    words = [word for word in text.split() if word.lower() in stopWords]
    if not words:
        return "Null"
    return " ".join(words)

In [9]:
title["stopwords"]=title.apply(lambda row: remove_stopwords(row["product_title"]),axis=1)

In [10]:
title

,product_title,stopwords
0,Multifunctional Air Compressor,Null
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with
...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for
4023,Update -3 Set Pressure Electric High Pressure ...,Null
4024,20V Tire Inflator Air Compressor Portable Bund...,with
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null


## Cleaning all titles

In [11]:
def cleaning(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)  ## http tags removal
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[\.;:,\?\"\'\/-]','',text)                            ## punctiations removal
    text=re.sub(r' {1,}',' ',text)                                        ## extra space removal
    words = []
    for word in text.split():
        word=word.lower()
        if word not in stopWords:                                        ## stopwords removal
            if word not in punctuation:
                word=nltk.stem.WordNetLemmatizer().lemmatize(word)       ## lemmatization

            words.append(word)
    return " ".join(words)

In [12]:
x="i am planning to go to hill station believes"
cleaning(x)

'planning go hill station belief'

In [13]:
## appending cleaned title to title dataframe

title["cleaned title"]=title.apply(lambda row: cleaning(row["product_title"]),axis=1)

In [14]:
title

,product_title,stopwords,cleaned title
0,Multifunctional Air Compressor,Null,multifunctional air compressor
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null,beverage air 312071b compressor 115v 60 hz r13...
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for,vbestlife air compressor spray gun， airbrush， ...
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with,ryobi p737d 18volt one+ cordless high pressure...
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with,2pcs air compressor cylinder head small 253p a...
...,...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for,professional portable airbrush black mini air ...
4023,Update -3 Set Pressure Electric High Pressure ...,Null,update 3 set pressure electric high pressure a...
4024,20V Tire Inflator Air Compressor Portable Bund...,with,20v tire inflator air compressor portable bund...
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null,hpdavv high pressure air compressor oilfree 15...


In [15]:
n_product=title.shape[0]
print(n_product)
title

4026


,product_title,stopwords,cleaned title
0,Multifunctional Air Compressor,Null,multifunctional air compressor
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null,beverage air 312071b compressor 115v 60 hz r13...
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for,vbestlife air compressor spray gun， airbrush， ...
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with,ryobi p737d 18volt one+ cordless high pressure...
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with,2pcs air compressor cylinder head small 253p a...
...,...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for,professional portable airbrush black mini air ...
4023,Update -3 Set Pressure Electric High Pressure ...,Null,update 3 set pressure electric high pressure a...
4024,20V Tire Inflator Air Compressor Portable Bund...,with,20v tire inflator air compressor portable bund...
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null,hpdavv high pressure air compressor oilfree 15...


In [20]:
title["cleaned title"][1]

'beverage air 312071b compressor 115v 60 hz r134a 240'

## Rapid Fuzz library

In [2]:
pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


In [16]:
from rapidfuzz import fuzz

In [17]:
s=title["cleaned title"].iloc[1]
s

'beverage air 312071b compressor 115v 60 hz r134a 240'

In [22]:
fuzz.token_sort_ratio(title["cleaned title"].iloc[0],title["cleaned title"].iloc[1])

39.02439024390244

## string2string matching

In [23]:
N=np.zeros(( n_product, n_product))
start=time.time()
for i in range( n_product):
    for j in range( n_product):
        N[i][j]=fuzz.token_sort_ratio(title["cleaned title"].iloc[i],title["cleaned title"].iloc[j])
end=time.time()
convert = str(datetime.timedelta(seconds = end-start))
print("Execution time: ",convert,"hrs: min: sec")

Execution time:  0:06:29.931154 hrs: min: sec


In [24]:
N

array([[100.        ,  39.02439024,  41.9047619 , ...,  42.99065421,
         28.41530055,  33.09352518],
       [ 39.02439024, 100.        ,  37.79527559, ...,  41.86046512,
         35.12195122,  31.05590062],
       [ 41.9047619 ,  37.79527559, 100.        , ...,  53.94736842,
         42.98245614,  44.56521739],
       ...,
       [ 42.99065421,  41.86046512,  53.94736842, ..., 100.        ,
         40.        ,  48.38709677],
       [ 28.41530055,  35.12195122,  42.98245614, ...,  40.        ,
        100.        ,  54.19847328],
       [ 33.09352518,  31.05590062,  44.56521739, ...,  48.38709677,
         54.19847328, 100.        ]])

## saving scores into dataframe

In [25]:
similarity=pd.DataFrame(N)
similarity.columns=title["product_title"].iloc[0:n_product]
similarity.index=title["product_title"].iloc[0:n_product]
similarity

product_title,Multifunctional Air Compressor,"Beverage Air 312-071B Compressor, 115V, 60 Hz, R134A 240",Vbestlife Air Compressor Spray Gun， Airbrush， Mini Air Compressor for Tattoo for Cake,Ryobi P737D 18-Volt ONE+ Cordless High Pressure Inflator with Digital Gauge & 18-Volt ONE+ Lithium-Ion 1.5 Ah LITHIUM+ Compact Battery (Bulk Packaged),"2Pcs Air Compressor Cylinder Head Small 2.5/3P Aluminum Cooling Parts 48mm x 62mm, Pneumatic Driving, with 2 Cylinder Heads","TBVECHI High Pressure Air Compressor Pump, 300BAR 4500PSI Electric Air Pump Airgun Scuba Rifle PCP 110V","Air pump XMYYHH Electric Bike Pump,Air Compressor,Portable Compressor Digital Car Tyre Pump,Rechargeable with LCD Display,for Car Bicycle Tires Balls Kayaking (Color : Black)","Modengzhe Portable Car Tire Inflator with Presure Gauge, DC 12V Car Cigarette Lighter Powered Mini Air Compressor Pump for Vehicles Motorcycles Bicycles Mopeds Inflatable Boats or Balls","Flexzilla Portable Air Compressor with Quiet Technology, 1 HP, 5 Gallon, Industrial Grade Pump",Campbell Hausfeld Campbell Hausfeld 2 Gal. Twin-Stack Air Compressor Kit,...,"425C 175 Psi Platinum Compressor Kit (Gen 2) W/Matte Black Filter, 1/4"" Port (12V, 33% Duty @ 175 Psi)",4 pics Air Compressor Belts PJ373 for H1504ST A700062 Husky Pumps,"beduan Brass Tire Inflator Lock on Air Chuck 1/4"" Hose Barb, Portable Compressor Tire inflator Valve Chuck Hose (Pack of 2)",760504007 Switch Replacement for Homelite OEM,Mini Portable 150PSI Car Tyre Air Compressor Pump 12V Electric Tire Inflator,Professional Portable Airbrush Black Mini Air Compressor Fashionable Design w/Built-in Holder for Makeup Tattoo Ink Hobby Decorating Commercial Arts,Update -3 Set Pressure Electric High Pressure Air Compressor Pump Machine- Airgun 30Mpa,20V Tire Inflator Air Compressor Portable Bundle with 20V Cordless Drill/Driver Kit,HPDAVV High Pressure Air Compressor Oil-Free 1.5 KW / 2 HP - 110v / 60Hz - 2.1cfm @ 4500 Psi - Scuba Tank/PCP Rife/Paintball Air Gun Filling Station Air Breathed Directly,CNCEST PCP Air Compressor 30MPA High Pressure Air Pump for Diving Pump Inflator Paintball Scuba Tank Filling Pump
product_title,,,,,,,,,,,,,,,,,,,,,
Multifunctional Air Compressor,100.000000,39.024390,41.904762,28.048780,32.394366,33.333333,27.513228,26.530612,39.655172,44.000000,...,35.000000,40.909091,33.802817,28.169014,45.283019,30.409357,38.260870,42.990654,28.415301,33.093525
"Beverage Air 312-071B Compressor, 115V, 60 Hz, R134A 240",39.024390,100.000000,37.795276,29.032258,34.146341,42.857143,23.696682,27.522936,31.884058,40.983607,...,39.436620,47.272727,35.365854,34.408602,40.625000,26.943005,37.956204,41.860465,35.121951,31.055901
Vbestlife Air Compressor Spray Gun， Airbrush， Mini Air Compressor for Tattoo for Cake,41.904762,37.795276,100.000000,34.449761,40.641711,49.717514,44.444444,42.323651,42.236025,46.896552,...,41.212121,42.105263,50.267380,27.586207,52.980132,53.703704,50.000000,53.947368,42.982456,44.565217
Ryobi P737D 18-Volt ONE+ Cordless High Pressure Inflator with Digital Gauge & 18-Volt ONE+ Lithium-Ion 1.5 Ah LITHIUM+ Compact Battery (Bulk Packaged),28.048780,29.032258,34.449761,100.000000,38.211382,38.135593,41.638225,49.333333,39.090909,38.235294,...,40.178571,33.333333,44.715447,24.000000,43.809524,42.909091,39.269406,45.497630,39.024390,45.267490
"2Pcs Air Compressor Cylinder Head Small 2.5/3P Aluminum Cooling Parts 48mm x 62mm, Pneumatic Driving, with 2 Cylinder Heads",32.394366,34.146341,40.641711,38.211382,100.000000,43.925234,42.066421,41.726619,43.434343,41.758242,...,43.564356,37.647059,42.857143,27.450980,46.808511,41.897233,46.700508,46.560847,43.018868,44.343891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Professional Portable Airbrush Black Mini Air Compressor Fashionable Design w/Built-in Holder for Makeup Tattoo Ink Hobby Decorating Commercial Arts,30.409357,26.943005,53.703704,42.909091,41.897233,42.798354,46.666667,49.511401,42.290749,41.706161,...,42.424

In [26]:
similarity.to_csv("I:/FLAIRMINDS/Product_matching/rapid_fuzz_str2str.csv")

## Top 10 similar product titles

In [27]:
def top_10__similar(p):
    x=similarity.iloc[p].values
    y=[]
    for j in range(n_product):
        y.append([x[j],j])
    y=y[1:]
    print(y[0:5])
    y.sort(key=lambda z:z[0],reverse=True)
    print("Query_Product_Title: ")
    print(Dp["product_url"][p],"-->",title["product_title"][p])
    print("*********************************************\n")
    print("Top 10 similar product title with similarity score are: \n")

    for i in range(10):
        print( f"{y[i][0]:.4f} --> ",Dp["product_url"][y[i][1]],"-->", title["product_title"][y[i][1]])

In [28]:
top_10__similar(55)

[[39.43661971830986, 1], [46.060606060606055, 2], [41.07142857142857, 3], [46.53465346534653, 4], [45.833333333333336, 5]]
Query_Product_Title: 
https://www.walmart.com/ip/Porter-Cable-PCFP3KIT-3-Piece-Nailer-and-0-8-HP-6-Gallon-Oil-Free-Pancake-Air-Compressor-Combo-Kit/910355347 --> Porter-Cable PCFP3KIT 3-Piece Nailer and 0.8 HP 6 Gallon Oil-Free Pancake Air Compressor Combo Kit
*********************************************

Top 10 similar product title with similarity score are: 

100.0000 -->  https://www.walmart.com/ip/Porter-Cable-PCFP3KIT-3-Piece-Nailer-and-0-8-HP-6-Gallon-Oil-Free-Pancake-Air-Compressor-Combo-Kit/910355347 --> Porter-Cable PCFP3KIT 3-Piece Nailer and 0.8 HP 6 Gallon Oil-Free Pancake Air Compressor Combo Kit
69.5652 -->  https://www.walmart.com/ip/Porter-Cable-CMB15-0-8-HP-1-5-Gallon-Oil-Free-Hand-Carry-Air-Compressor-Kit/21999547 --> Porter-Cable CMB15 0.8 HP 1.5 Gallon Oil-Free Hand Carry Air Compressor Kit
67.1329 -->  https://www.amazon.com/dp/B07WF7CNZT -->